In [ ]:
import copclib as copc
import numpy as np
import time

import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
# Datei Einlesen:
reader = copc.FileReader("./daten/autzen.copc.laz")
#reader = copc.FileReader("C:/Users/vlasi/Downloads/sofi.copc.laz")

# Copc Info und Las Header (für boxen)
config = reader.copc_config
copc_info = config.copc_info
las_header = config.las_header

# Bounding Box der Root node extrahieren: 

    # Root Voxelkey anlegen
root_key = copc.VoxelKey(0,0,0,0)

    # Root Entry (Node) laden (ohne Punkte)
root_node = reader.FindNode(root_key)

    # Bounding Box konstruieren aus VoxelKey und Las-Header
root_box = copc.Box(root_key, las_header)
print("Root Box:\n\txmin:{}\n\txmax:{}\n\n\tymin:{}\n\tymax:{}\n\n\tzmin:{}\n\tzmax:{}".format(root_box.x_min, root_box.x_max, root_box.y_min, root_box.y_max, root_box.z_min, root_box.z_max))


In [ ]:
# Alle Nodes auslesen und Bounding Boxen konstruieren

all_entries = reader.GetAllNodes()
boxes = []

for entry in all_entries:
    boxes.append(copc.Box(entry.key, las_header)) 

In [ ]:
from visuals import draw_boxes
from ray import Ray

# Erstelle Instanz eines Strahles
#Strahl2 = Ray(np.array([376600, 3.7587e6, 1300]), np.array([-1, -2, -2]))
Strahl = Ray(np.array([640000, 855000, 5100]), np.array([-1, -2, -2]))

#Erstelle Eine Grafik mit mpl und den zwei Hilfsfunktionen
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

draw_boxes(boxes, ax)
Strahl.draw_ray(ax, 8000)

In [ ]:
# Intersection test mit allen Nodes
intersected_nodes = []
intersected_boxes = []

start = time.time()
for entry in all_entries:
    box = copc.Box(entry.key, las_header)
    if Strahl.slab_test(box):
        intersected_nodes.append(entry)
        intersected_boxes.append(box)

dauer = time.time()-start
dauer

In [ ]:
intersected_nodes

In [ ]:
# Plotten der geschnittenen Nodes
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

draw_boxes(intersected_boxes, ax)
Strahl.draw_ray(ax, 9000)

In [ ]:
# Punkte aus den geschnittenen Nodes extrahieren
intersected_points_packed = []
intersected_points = []

for node in intersected_nodes:
    intersected_points_packed.append(reader.GetPoints(node))

for points in intersected_points_packed:
    for point in points:
        intersected_points.append(point)

# Beispiele für die point / points Klassen
beispielpunkt = intersected_points_packed[0][0]
beispielpunkte = intersected_points_packed[0]

intersected_points

In [ ]:
# Beispiel für rekursives traversieren der Nodes
# Hier um alle Nodes mit Punkten zu finden
import node_traversal

nodes = node_traversal.get_all_nodes("./daten/autzen.copc.laz")
#nodes = node_traversal.get_all_nodes("/home/vlasi/Downloads/sofi.copc.laz")

print(f"{len(nodes)} nodes Mit Punkten gefunden:")
for node_key in nodes[:10]:
    node_info = reader.FindNode(node_key)
    print(f"  - Key: {node_key}, Point Count: {node_info.point_count}")